In [1]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Mouse C3KO-RichLi snRNA
####  2023-05-22 by Yi Zhao (Texas Heart Institute, US)
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Initiate  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
Ver <- '0'
Step <- 'PART23_Cell_Type_DEG'
Project <- '2023_neoc3ko_rli'

Code_dir <- paste0('/Volumes/shire/project/', Project, '/code/mouse_v', Ver, '/')

source(paste0(Code_dir, 'src/bioinformatics.R'))
source(paste0(Code_dir, 'src/scRNAseq.R'))
source(paste0(Code_dir, 'src/scATACseq.R'))
source(paste0(Code_dir, 'mouse_v', Ver, '.helper_functions.R'))
suppressMessages(library('DESeq2'))

InitiateProject('Rivendell', Ver, Step, 'mouse', Project, 'shire')
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [2]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Load data  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
full.srt <- readRDS('integrated/PART19.annotated.srt.rds')
#full.srt@meta.data$group1 <- droplevels(full.srt@meta.data$group1)

In [3]:
Table(full.srt$Cell_type)


Cardiomyocyte   Endocardium    Epicardium    Fibroblast        Immune 
        56287         13462          6665          9715          2187 
     Pericyte     Adipocyte           SMC           BEC           LEC 
         1723           405          1068          3007           349 
Contamination       Doublet 
          402          6365 

In [4]:
Table(full.srt$group2)


wt_p2m wt_p2s c3_p2m c3_p2s 
 28221  22628  25319  25467 

In [5]:
Table(full.srt$group1)


wt_p2m1 wt_p2m2 wt_p2s1 wt_p2s2 c3_p2m1 c3_p2m2 c3_p2s1 c3_p2s2 
  17937   10284   12176   10452   14964   10355   19076    6391 

In [6]:
full.srt <- full.srt[, !full.srt$Cell_type %in% c('Doublet','Contamination')]

## call the marker of each cell type

In [7]:
### cell type marker
PlotPDF('1.heat.cell_type_marker', 12, 15)

        Idents(full.srt) <- 'Cell_type'
        mk <- FindAllMarkers(full.srt, logfc.threshold = 0.25, only.pos = T, return.thresh = 0.05)
        mk <- mk[mk$p_val_adj < 0.05, ]
        print(MarkerHeatmap(full.srt, mk, top = 10, disp.min = 0, raster = T))
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        saveRDS(mk, 'analysis/PART23.cell_type_marker.srt_mk.rds')
        write.table(mk,file = "analysis/PART23.cell_type_marker.srt_mk.txt",quote = FALSE, sep = "\t")
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dev.off()
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Calculating cluster Cardiomyocyte

Calculating cluster Endocardium

Calculating cluster Epicardium

Calculating cluster Fibroblast

Calculating cluster Immune

Calculating cluster Pericyte

Calculating cluster Adipocyte

Calculating cluster SMC

Calculating cluster BEC

Calculating cluster LEC

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


pdf 
  2

## MarkerJitterPlot: LatsCKO vs Control

In [12]:
MarkerJitterPlot <- function(mk.df, cols = 'GnBu', col.direction = 0, jitter.width = 0.3, pt.size = 0.5){
        mk.df$TMP____logp <- -log10(mk.df$p_val_adj)
        mk.df$TMP____logp[mk.df$TMP____logp > 100] <- 100
        ggplot(mk.df, aes(x = cluster, y = avg_log2FC)) +
                geom_jitter(width = jitter.width, aes(color = TMP____logp), size = pt.size) +
                scale_color_distiller(palette = cols, direction = col.direction) +
                labs(x = 'Clusters', y = 'Mean Log2FC', color = '-Log10 Adj.P') +
                theme_classic() +
                theme(aspect.ratio = 1, axis.text.x = element_text(hjust = 1, angle = 45))
}

In [13]:
srt_list <- SplitObject(full.srt, 'Cell_type')
ct <- names(srt_list)
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Compute sc LatsCKO vs Control DEG  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
plan("multisession", workers = 8)
sc_deg_list <- list()
for(i in 1:L(ct)){
        message(ct[i])
        Idents(srt_list[[i]]) <- 'group2'
        deg <- FindMarkers(srt_list[[i]], ident.1 = 'c3_p2m', ident.2 = 'wt_p2m')
        deg <- deg[deg$p_val_adj < 0.05, ]
        deg$gene <- rownames(deg)
        sc_deg_list[[i]] <- deg
        names(sc_deg_list)[i] <- ct[i]
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        saveRDS(sc_deg_list, 'analysis/PART23.sc_deg_c3_p2m_vs_wt_p2m.srt_mk.rds')
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
}
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epicardium

Immune

Cardiomyocyte

SMC

Endocardium

Fibroblast

Pericyte

Adipocyte

BEC

LEC



In [14]:
cell_state_mk <- sc_deg_list
celltype <- names(cell_state_mk)
mk.df <- data.frame()

for (i in 1:L(ct)){
    #names(cell_state_mk)
    #print(celltype[i])
    cell_state_mk[[i]]$cluster <- celltype[i]
    mk.df <- rbind(mk.df,cell_state_mk[[i]])
}

In [15]:
write.table(mk,file = "analysis/PART23.cell_c3_p2m_vs_wt_p2m_marker.txt",quote = FALSE, sep = "\t")


In [16]:
Table(mk.df$cluster)


    Adipocyte           BEC Cardiomyocyte   Endocardium    Epicardium 
           82           297           841           472           538 
   Fibroblast        Immune           LEC      Pericyte           SMC 
          338           111            10           239           104 

In [17]:
p1 <- MarkerJitterPlot(mk.df = mk.df)
PlotPDF('2.0.Jitter.cell_c3_p2m_vs_wt_p2m_marker.0',10, 5)
p1
dev.off()

pdf 
  2

In [32]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Compute pseudobulk DEG  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
full.srt <- full.srt[, full.srt$group2 %in% c('c3_p2m', 'wt_p2m')]
full.srt@meta.data$group1 <- droplevels(full.srt@meta.data$group1)
full.srt@meta.data$group2 <- droplevels(full.srt@meta.data$group2)

srt_list <- SplitObject(full.srt, 'Cell_type')
ct <- names(srt_list)

psb_deg_list <- list()
for(i in 1:L(ct)){
        message(ct[i])
        #srt_list[[i]] <- RenameAssays(srt_list[[i]], 'CBN' = 'RNA')
        psb <- AggregateExpression(srt_list[[i]], assays = 'RNA', slot = 'count', group.by = 'group1')$RNA
        meta <- srt_list[[i]]@meta.data[!duplicated(srt_list[[i]]$group1), ]
        rownames(meta) <- meta$group1
        meta <- meta[colnames(psb), ]
        ##DESEQ2 workflow
        dds <- DESeqDataSetFromMatrix(psb, colData = meta, design = ~ group2)
        dds <- estimateSizeFactors(dds)
        dds <- DESeq(dds)
        res <- results(dds, alpha = 0.05, independentFiltering = T, contrast = c('group2', 'c3_p2m', 'wt_p2m'))
        res <- lfcShrink(dds, contrast = c('group2', 'c3_p2m', 'wt_p2m'), res = res, type = "ashr")
        deg_up <- rownames(res)[res$padj < 0.05 & res$log2FoldChange > 0]
        deg_dn <- rownames(res)[res$padj < 0.05 & res$log2FoldChange < 0]
        deg_up <- deg_up[!is.na(deg_up)]
        deg_dn <- deg_dn[!is.na(deg_dn)]
        psb_deg_list[[i]] <- list('UP' = deg_up, 'DN' = deg_dn)
        names(psb_deg_list)[i] <- ct[i]
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        saveRDS(psb_deg_list, 'analysis/PART22.psb_deg_c3_p2m_vs_wt_p2m.list.rds')
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
}
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Intersect sc and psb DEGs  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
final_deg_list <- list()
for(i in 1:L(ct)){
        message(ct[i])
        sc_deg <- sc_deg_list[[i]]
        final_deg_list[[i]] <- sc_deg[(sc_deg$gene %in% psb_deg_list[[i]][['UP']] & sc_deg$avg_log2FC > 0) |
                                              (sc_deg$gene %in% psb_deg_list[[i]][['DN']] & sc_deg$avg_log2FC < 0), ]
        gl <- split(final_deg_list[[i]]$gene, final_deg_list[[i]]$avg_log2FC > 0)
        print(str(gl))
        names(final_deg_list)[i] <- ct[i]
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        saveRDS(final_deg_list, 'analysis/PART22.final_deg_tsc_vs_control.srt_mk.rds')
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
}
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epicardium

converting counts to integer mode

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

Immune

converting counts to integer mode

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

Cardiomyocyte

converting counts to integer mode

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-disper

List of 2
 $ FALSE: chr [1:56] "C3" "Trim12a" "H2-D1" "H2-K1" ...
 $ TRUE : chr [1:76] "Gm47101" "Mmp2" "Tmcc1" "Prex2" ...
NULL


Immune



List of 2
 $ FALSE: chr [1:6] "Hba-a2" "Trim12a" "Trim30d" "Hbb-bs" ...
 $ TRUE : chr [1:3] "Cd81" "Nppa" "Postn"
NULL


Cardiomyocyte



...

 Named list()
NULL


BEC



List of 2
 $ FALSE: chr [1:5] "Cdk14" "Hjurp" "Hdac9" "Gm42047" ...
 $ TRUE : chr [1:52] "Mctp1" "Ctsl" "Ppib" "Fn1" ...
NULL


LEC



List of 1
 $ TRUE: chr "Pdpn"
NULL


## MarkerJitterPlot

In [38]:
MarkerJitterPlot <- function(mk.df, cols = 'GnBu', col.direction = 0, jitter.width = 0.3, pt.size = 0.5){
        mk.df$TMP____logp <- -log10(mk.df$p_val_adj)
        mk.df$TMP____logp[mk.df$TMP____logp > 300] <- 300
        ggplot(mk.df, aes(x = cluster, y = avg_log2FC)) +
                geom_jitter(width = jitter.width, aes(color = TMP____logp), size = pt.size) +
                scale_color_distiller(palette = cols, direction = col.direction) +
                labs(x = 'Clusters', y = 'Mean Log2FC', color = '-Log10 Adj.P') +
                theme_classic() +
                theme(aspect.ratio = 1, axis.text.x = element_text(hjust = 1, angle = 45))
}

In [34]:
cell_state_mk <- readRDS(file = 'analysis/PART22.final_deg_tsc_vs_control.srt_mk.rds')
celltype <- names(cell_state_mk)
mk.df <- data.frame()

for (i in 1:6){
    #names(cell_state_mk)
    #print(celltype[i])
    cell_state_mk[[i]]$cluster <- celltype[i]
    mk.df <- rbind(mk.df,cell_state_mk[[i]])
}

In [35]:
Table(mk.df$cluster)


Cardiomyocyte   Endocardium    Epicardium    Fibroblast        Immune 
          411           280           132            73             9 
          SMC 
            2 

In [36]:
min(abs(mk.df$avg_log2FC))

[1] 0.2506521

In [39]:
p1 <- MarkerJitterPlot(mk.df = mk.df)
PlotPDF('03.Jitter.cell_state_marker.final',6, 5)
p1
dev.off()

pdf 
  2

In [10]:
cell_state_mk <- readRDS(file = 'analysis/PART22.cell_state_marker.srt_mk.rds')
names(cell_state_mk)
celltype <- names(cell_state_mk)
mk.df <- data.frame()

for (i in 1:6){
    #names(cell_state_mk)
    #print(celltype[i])
    cell_state_mk[[i]]$cluster <- celltype[i]
    mk.df <- rbind(mk.df,cell_state_mk[[i]])
}

[1] "EX"         "IN"         "VLMC_Endo"  "Astro"      "Micro"     
[6] "Oligo_OPCs"

In [11]:
Table(mk.df$cluster)


     Astro         EX         IN      Micro Oligo_OPCs  VLMC_Endo 
      2272       5544       2942       4284       6775       4088 

In [12]:
p2 <- MarkerJitterPlot(mk.df = mk.df)
PlotPDF('2.Jitter.cell_state_marker',6, 5)
p2
dev.off()

pdf 
  2

In [ ]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Calculate cell state markers for each cell type  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
plan("multisession", workers = 8)
cell_state_mk <- list()

PlotPDF('1.heat.cell_state_marker', 10, 10)
for(i in 1:L(ct)){
        message(ct[i])
        Idents(srt_list[[i]]) <- 'Cell_state'
        mk <- FindAllMarkers(srt_list[[i]], logfc.threshold = 0.25, only.pos = T, return.thresh = 0.05)
        mk <- mk[mk$p_val_adj < 0.05, ]
        print(MarkerHeatmap(srt_list[[i]], mk, top = 10, disp.min = 0, raster = T))
        cell_state_mk[[i]] <- mk
        names(cell_state_mk)[i] <- ct[i]
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        saveRDS(cell_state_mk, 'analysis/PART22.cell_state_marker.srt_mk.rds')
        ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
}
dev.off()
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~